In [ ]:
file = open("./py_temp/test1.txt","w")
file.write(" 텍스트 파일에 처음 쓴 글입니다")
file.close( )

In [ ]:
file2 = open("./py_temp/test1.txt","w")
file2.write("텍스트 파일에 두번째 쓴 글입니다")
file2.close( )

In [ ]:
file3 = open("./py_temp/test1.txt","a")
file3.write("텍스트 파일에 세번째 쓴 글입니다")
file3.close( )

In [ ]:
file4 = open("./py_temp/test1.txt","a")
file4.write("\n" + "텍스트 파일에 네번째 쓴 글입니다")
file4.close( )

In [ ]:
import sys

print('파이썬이 재미있다고 첫번째 쓴 글입니다')
orig_stdout = sys.stdout
file5 = open("./py_temp/test1.txt","a")
sys.stdout = file5  #모니터에 출력하지 말고 file 에 출력해라

print()
print('파이썬이 재미있다고 두번째 쓴 글입니다')

sys.stdout = orig_stdout  #원래대로 변경 - 다시 화면에 출력시켜라    
print('저장이 완료되었습니다')

In [ ]:
file5.close()

In [1]:
import pandas as pd

# 표 ( 데이터 프레임 ) 만들기
no = [ ]
subject_name = [ ]

no.append(1)
no.append(2)
no.append(3)

subject_name.append('수학')
subject_name.append('과학')
subject_name.append('빅데이터')

subject = pd.DataFrame( )
subject['과목번호'] = no
subject['과목명'] = subject_name
print(subject)


   과목번호   과목명
0     1    수학
1     2    과학
2     3  빅데이터


In [ ]:
subject.head()

In [5]:
writer = pd.ExcelWriter("./py_temp/test1.xlsx")
subject.to_excel(writer, sheet_name='sheet1', index=False)
writer.close()

In [ ]:
!pip install openpyxl

In [11]:
# csv 형식으로 저장하기
subject.to_csv("./py_temp/test1.csv", encoding="utf-8-sig",index=False)

# xls 형식으로 저장
subject.to_excel("./py_temp/test1.xlsx" , engine='openpyxl' , index=False)


In [ ]:
subject.to_excel("./py_temp/test1.xlsx" , engine='openpyxl' , index=True)

In [12]:
#대한민국 구석구석 사이트 상세 정보 수집한 후 다양한 형식으로 저장하기
#Step 1. 필요한 모듈을 로딩합니다
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time          
import pandas as pd    
import math
import os

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 대한민국 구석구석 사이트 정보 수집용 웹크롤러입니다.")
print("=" *100)

query_txt = input('1.정보를 수집할 키워드는 무엇입니까?: ')

cnt = int(input('2.몇 건의 정보를 수집할까요? :'))
page_cnt = math.ceil( cnt / 10 )
print('총 %s 건의 정보를 수집하기 위해 %s 페이지까지 이동할 예정입니다' %(cnt , page_cnt))

#Step 3. 수집된 데이터를 저장할 폴더와 파일명 지정하기
f_dir = input("3.파일을 저장할 폴더명만 쓰세요(기본값:./py_temp/):")
if f_dir == '' :
    f_dir="./py_temp/"

# 결과를 저장할 폴더와 파일명 지정    
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

if os.path.exists(f_dir) :
    print('지정한 %s 디렉토리가 존재하여 해당 디렉토리 아래에 파일을 생성합니다' %f_dir)
else :
    print('지정한 %s 디렉토리가 존재하지 않아 새로 생성합니다' %f_dir)
    os.makedirs(f_dir)
    
result_dir= f_dir+'대한민국구석구석'+'-'+s+'-'+query_txt 
os.makedirs(result_dir)
ft_name = result_dir+'/'+'대한민국구석구석'+'-'+s+'-'+query_txt+'.txt'
fc_name = result_dir+'/'+'대한민국구석구석'+'-'+s+'-'+query_txt+'.csv'
fx_name = result_dir+'/'+'대한민국구석구석'+'-'+s+'-'+query_txt+'.xlsx'
    
print("\n")    

#Step 4. 검색어 입력한 후 자동 검색하기
s = Service("./116/chromedriver.exe")
driver = webdriver.Chrome(service=s)

query_url = 'https://korean.visitkorea.or.kr/'

driver.get(query_url)
time.sleep(2)
driver.maximize_window()

s_time = time.time( )
element = driver.find_element(By.ID,"inp_search")
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(5)

# Step 5. 본문 내역 추출하기
time.sleep(0.2)
s_time = time.time( ) 

no2=[]           # 게시글 번호 컬럼
title2=[ ]       # 게시글 제목 컬럼
org2=[]          # 지자체명 컬럼
contents2=[ ]     # 본문 내용 컬럼
tag2=[ ]         # 해시태그 컬럼

no = 1           # 게시글 번호 초기값

for a in range(1,page_cnt+1) :
    print('')
    print('현재 %s 페이지에 있는 정보를 수집합니다' %a)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    content_list = soup.find('ul','list_thumType type1').find_all('li')
    cnt_no = 1
    
    for b in content_list:       
        try :
            title = b.find('div','area_txt').find('div','tit').get_text().strip()
        except :
            continue
        else :
            # 1. 게시글 번호
            print("\n")
            print("%s 번째 정보를 추출하고 있습니다============" %no)
            no2.append(no)
            print("1.번호 : %s" %no)

            # 2. 게시글 제목
            title2.append(title)
            print("2.제목 : %s" %title )
            
            # 3. 지자체이름
            org = b.find('div','service').find('p').get_text().strip()
            org2.append(org)
            print("3.지자체이름: %s" %org)
            
            #4. 해시태그
            try :
                tag = b.find('p','tag_type').get_text().replace("#"," #")
            except :
                tag = ' '             
            tag2.append(tag)
            print("4.해시태그: %s" %tag)
            
            #5. 본문   
            # 1 페이지에만 광고가 들어 있어서 xpath 값이 다름
            # 2 페이지부터는 광고가 없어서 1씩 증가시키면 됨
            if a == 1 and cnt_no == 5 :
                cnt_no += 1
                
            driver.find_element(By.XPATH,'//*[@id="listBody"]/ul/li[%s]/div[2]/div[1]/a' %cnt_no).click()
            cnt_no += 1
            
            time.sleep(5)
            
            html_2 = driver.page_source
            soup_2 = BeautifulSoup(html_2, 'html.parser')
            
            content1 = soup_2.find('div','inr_wrap')
            content2 = soup_2.find('div','box_txtPhoto')
            
            if content1 :
                content = content1.find('div','inr').find('p').get_text()
                contents2.append(content)
                print('5.본문내용:', content)
            elif content2 :
                content_1 = soup_2.find('div','box_txtPhoto')
                try :
                    content_1.style.decompose()
                    content_1.script.decompose()
                except :
                    content_2 = content_1.find_all('div','txt_p')
                    print('5.본문내용:') 
                    for b in content_2 :
                        content = b.get_text().strip()
                        print(content)
                        contents2.append(content)
                else :
                    content_2 = content_1.find_all('div','txt_p')
                    print('5.본문내용:') 
                    for b in content_2 :
                        content = b.get_text().strip()
                        print(content)
                        contents2.append(content)
            
            # 수집된 내용을 txt 형식으로 저장하기
            f = open(ft_name , 'a' , encoding='UTF-8')
            f.write('\n'+'1.번호:' + str(no))
            f.write('\n'+'2.제목:' + title)
            f.write('\n'+'3.지자체이름:'+org)
            f.write('\n'+'4.해시태그:'+tag)
            f.write('\n'+'5.본문내용:'+content+'\n')
            f.close( )
                       
            driver.back()
            
            time.sleep(3)
                       
            no += 1         # 전체 게시글 번호용 값
            
            if no > cnt :
                 break
                
            time.sleep(0.5)
            
    a += 1
    try :
        driver.find_element(By.LINK_TEXT,'%s' %a).click()
    except :
        driver.find_element(By.LINK_TEXT,'다음').click()

    time.sleep(0.5)

# Step 6. 결과를 데이터프레임으로 변환 후 저장하기
df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['지자체명']=pd.Series(org2)
df['본문내용']=pd.Series(contents2)
df['해시태그']=pd.Series(tag2)

# xls , csv 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8", engine='openpyxl')
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

e_time = time.time( )     
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("1.크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집 완료 했습니다" %(cnt,no-1))
print('2.txt 파일 저장경로: ',ft_name)
print('3.xlsx 파일 저장경로: ',fx_name)
print('4.csv 파일 저장경로: ',fc_name)
print("5.요청하신 작업의 총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("=" *80)

driver.close( )

 이 크롤러는 대한민국 구석구석 사이트 정보 수집용 웹크롤러입니다.
1.정보를 수집할 키워드는 무엇입니까?: 수원 화성
2.몇 건의 정보를 수집할까요? :21
총 21 건의 정보를 수집하기 위해 3 페이지까지 이동할 예정입니다
3.파일을 저장할 폴더명만 쓰세요(기본값:./py_temp/):
지정한 ./py_temp/ 디렉토리가 존재하여 해당 디렉토리 아래에 파일을 생성합니다




ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=116.0.5845.141)
Stacktrace:
	GetHandleVerifier [0x00B537C3+48947]
	(No symbol) [0x00AE8551]
	(No symbol) [0x009EC7F0]
	(No symbol) [0x00A16458]
	(No symbol) [0x00A15AB5]
	(No symbol) [0x00A3508C]
	(No symbol) [0x00A12EB6]
	(No symbol) [0x00A353E4]
	(No symbol) [0x00A475DA]
	(No symbol) [0x00A34E86]
	(No symbol) [0x00A116C7]
	(No symbol) [0x00A1284D]
	GetHandleVerifier [0x00D9FDF9+2458985]
	GetHandleVerifier [0x00DE744F+2751423]
	GetHandleVerifier [0x00DE1361+2726609]
	GetHandleVerifier [0x00BD0680+560624]
	(No symbol) [0x00AF238C]
	(No symbol) [0x00AEE268]
	(No symbol) [0x00AEE392]
	(No symbol) [0x00AE10B7]
	BaseThreadInitThunk [0x76B37BA9+25]
	RtlInitializeExceptionChain [0x7768B79B+107]
	RtlClearBits [0x7768B71F+191]
